In [ ]:
#Erick Barron
#03/17/2024
#Comp 541 Prof. Klotzman
#environment setup:
#import necessary items
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

#Setup and Inital Exploration Cell 1

#Data loading:
PizzaSales = pd.read_csv(r"C:\Code\CodeExamples\Comp541\pizza_sales.csv\pizza_sales.csv")

#Prelim data exploration:
#inspect, summarize, and check for missing values
print(PizzaSales.head())
print(PizzaSales.info())
PizzaSales.describe()
PizzaSales.isnull().sum()


In [ ]:
#Data Cleaning and Prep Cell 2

#Datetime conversion:
#show columns
print(PizzaSales.columns)
#remove space from column titles so I can apply titles without spaces
PizzaSales.columns = PizzaSales.columns.str.strip()

#create formats for both types that are under order_date
date_formats = ['%m/%d/%Y %H:%M:%S', '%d-%m-%Y %H:%M:%S']

#combine date and time to create datetime
datetime_column = pd.to_datetime(PizzaSales['order_date'] + ' ' + PizzaSales['order_time'], errors='coerce')

#update datetime column with both formats
for fmt in date_formats:
    datetime_column.fillna(pd.to_datetime(PizzaSales['order_date'] + ' ' + PizzaSales['order_time'], format=fmt, errors='coerce'), inplace=True)

#assign datetime column with process above
PizzaSales['datetime_column'] = datetime_column

#drop combined columns
PizzaSales.drop(['order_date', 'order_time'], axis=1, inplace=True)

#Data type optimization: change pizza and order id since they increase every row and get very large
#define columns with data
data_types = {
    'pizza_id': int,
    'order_id': int,
}

#convert large int to smaller numeric types
for col in data_types.keys():
    PizzaSales[col] = pd.to_numeric(PizzaSales[col], downcast='unsigned')

#Handle missing data/duplication:
#although no missing values noted it seems pizza_name_id and pizza_name are redundant so will drop one
PizzaSales.drop(columns=['pizza_name'], inplace=True)

#Final Dataset Prep:
#print again to see how the updates affect the data
print(PizzaSales.columns)
print(PizzaSales.head())
print(PizzaSales.info())
PizzaSales.describe()
PizzaSales.isnull().sum()






In [ ]:
#Exploratory Data Analysis Cell 3

#Sales Trends Analysis:

#set datetime as column index
PizzaSales.set_index('datetime_column', inplace=True)

#resample the data for daily, weekly, and monthly
DailySales = PizzaSales['total_price'].resample('D').sum()
WeeklySales = PizzaSales['total_price'].resample('W').sum()
MonthlySales = PizzaSales['total_price'].resample('M').sum()

#print rows of resampled data to see
print("Daily Sales:")
print(DailySales.head())
print("Weekly Sales:")
print(WeeklySales.head())
print("Monthly Sales:")
print(MonthlySales.head())

#plot daily sales
plt.figure(figsize=(10, 6))
DailySales.plot(color = 'red')
plt.title('Daily Pizza Sales', fontsize = 20, color = 'green')
plt.xlabel('Date', fontsize = 16, color = 'green')
plt.ylabel('Total Sales', fontsize = 16, color = 'green')
plt.show()

#plot weekly sales
plt.figure(figsize=(10, 6))
WeeklySales.plot(color = 'red')
plt.title('Weekly Pizza Sales', fontsize = 20, color = 'green')
plt.xlabel('Date', fontsize = 16, color = 'green')
plt.ylabel('Total Sales', fontsize = 16, color = 'green')
plt.show()

#plot monthly sales
plt.figure(figsize=(10, 6))
MonthlySales.plot(color = 'red')
plt.title('Monthly Pizza Sales', fontsize = 20, color = 'green')
plt.xlabel('Date', fontsize = 16, color = 'green')
plt.ylabel('Total Sales', fontsize = 16, color = 'green')
plt.show()


#Performance By Category and Size:

#plot for category
plt.figure(figsize=(10, 6))
sns.countplot(x='pizza_category', data=PizzaSales, color = 'red')
plt.title('Pizza Sales by Category', fontsize = 20, color = 'green')
plt.xlabel('Pizza Category', fontsize = 16, color = 'green')
plt.ylabel('Total Sales', fontsize = 16, color = 'green')
plt.show()

#plot for size
plt.figure(figsize=(10, 6))
sns.countplot(x='pizza_size', data=PizzaSales, color = 'red')
plt.title('Pizza Sales by Size', fontsize = 20, color = 'green')
plt.xlabel('Pizza Size', fontsize = 16, color = 'green')
plt.ylabel('Total Sales', fontsize = 16, color = 'green')
plt.show()

#Popularity Analysis:

#color for charts
PizzaPalette = ["#FF2600", "#FFD700", "#228B22", "#8B4513", "#556B2F"]

#total sales
pizza_total_sales = PizzaSales.groupby('pizza_name_id')['total_price'].sum().sort_values(ascending=False)
#frequency of order
pizza_order_count = PizzaSales['pizza_name_id'].value_counts().sort_values(ascending=False)
#average quantity
pizza_avg_quantity = PizzaSales.groupby('pizza_name_id')['quantity'].mean().sort_values(ascending=False)



# Bar plot for total sales
plt.figure(figsize=(20, 20))
sns.barplot(x=pizza_total_sales.values, y=pizza_total_sales.index, palette=PizzaPalette, legend=False)
plt.title('Total Sales by Pizza')
plt.xlabel('Total Sales')
plt.ylabel('Pizza Name')
plt.tight_layout()
plt.show()
# Bar plot for frequency of orders
plt.figure(figsize=(20, 20))
sns.barplot(x=pizza_order_count.values, y=pizza_order_count.index, palette=PizzaPalette)
plt.title('Order Count by Pizza')
plt.xlabel('Order Count')
plt.ylabel('Pizza Name')
plt.show()
# Bar plot for average quantity ordered
plt.figure(figsize=(20, 20))
sns.barplot(x=pizza_avg_quantity.values, y=pizza_avg_quantity.index, palette=PizzaPalette)
plt.title('Average Quantity Ordered by Pizza')
plt.xlabel('Average Quantity Ordered')
plt.ylabel('Pizza Name')
plt.show()

#Ingredients Analysis:

#get ingredients
ingredients = PizzaSales['pizza_ingredients'].str.split(',')

#transform to flatten list to work with ingredients list
#iterates outer loop and inner loop to find ingredients and strip helps with spacing discrepancies
IngredientsList = [ingredient.strip() for sublist in ingredients for ingredient in sublist]

#count number of times ingredient appears
NumIngredients = pd.Series(IngredientsList).value_counts()

#create bar chart for the top ten popular ingredients
plt.figure(figsize=(10, 6))
NumIngredients.head(10).plot(kind='bar')
plt.title('Top 10 Most Popular Ingredients')
plt.xlabel('Ingredient')
plt.ylabel('Amount')
plt.show()

In [ ]:
#Recommendation System Development Cell 4

#Data Prep:

#remove spaces from the 'pizza_name_id' column since they are so varied
PizzaSales['pizza_name_id'] = PizzaSales['pizza_name_id'].str.strip()

#prep data normally load user data but we will load necessary columns and create matrix
TransactionData = PizzaSales[['order_id', 'pizza_name_id', 'quantity']]
TransactionMatrix = TransactionData.pivot_table(index='order_id', columns='pizza_name_id', values='quantity', fill_value=0)


#Correlation Analysis:
#this uses pearson correlation between the columns loaded in previous methods
CorrelationMatrix = TransactionMatrix.corr()

#Recomendation Algorithm:
#content based filtering however I did not create users as
#the similarities will be based on pizza similarities not user preferences
def recommend_items(PizzaChoice, CorrelationMatrix, n=5):
    similar_items = CorrelationMatrix[PizzaChoice].sort_values(ascending=False)[1:n+1]
    return similar_items.index.tolist()

#System Evaluation:
#use by changing PizzaChoice to something under the pizza_name_id column
#uses recomendation algorithm
PizzaChoice = 'hawaiian_m'
recommended_items = recommend_items(PizzaChoice, CorrelationMatrix)

#plot recommended items (this looks better than the bland output :)) Invert for top recommendation on top
plt.figure(figsize=(10, 6))
plt.barh(recommended_items, width=1, color='dodgerblue')
plt.xlabel('Recommendation Score')
plt.ylabel('Recommended Pizzas')
plt.title('Recommended Pizzas For ' + PizzaChoice)
plt.gca().invert_yaxis()
plt.show()


#calculate cosine similarity 
#here we do normalize the data
def cosine_similarity_manual(matrix):
    norm_matrix = matrix / np.linalg.norm(matrix, axis=0)
    similarity_matrix = np.dot(norm_matrix.T, norm_matrix)
    return similarity_matrix

#calculate cosine similarity after getting the necessary data from csv
cosine_sim = cosine_similarity_manual(TransactionMatrix.values)

#plot cosine similarity matrix (need to expand to see all)
#shows similarity between pizza types with 1 being exact and -1 being opossite although this will not show up here I believe
plt.figure(figsize=(50, 50))
sns.heatmap(cosine_sim, cmap='viridis', annot=True, fmt='.2f', xticklabels=TransactionMatrix.columns, yticklabels=TransactionMatrix.columns)
plt.title('Cosine Similarity of Pizzas')
plt.xlabel('Pizza Name')
plt.ylabel('Pizza Name')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.show()


#Insights and Actionable Recomendations Cell 5 
Key Insights: For insights the charts and graphs are the most important for findings. Such as finding what exactly is the most popular and when it is best to sell and how. Finding the most popular pizzas and toppings will also provide key insights and the visualization helps with the clarity that it provides. Although the graphs for best selling from daily, weekly, to monthly are a bit hard to read since I could not get the dates to update so they all print out monthly however enlarging and studying gives key insights into popular times for selling. 

Actionable Recommendations: Given the data many actions and recommendations may be given. The sales data may give options for when to post sales, close, or order less ingredients to save money. Another recommendation may be to see the most popular ingredients and combine into new pizzas according to the data a garlic and tomatoes pizza may be popular(although in this case tomatoes may just be sauce). The sales and order by pizza may allow you to downsize the menu and with the help of the pizza by category chart allow you to create pizzas by group and find new combinations and make the menu more readable. Recommendation systems can give you a marketing strategy to release limited time pizza to keep getting return customers while also saving buying in season ingredients. Lastly finding the similarity of pizzas will allow you to remove pizzas that are too similar or maybe offer more options with pizzas that have ingredient overlap and give customers more options and thus netting a larger range of customers. ALthough personally a make you own is good for a pizza shop anyway.


#Conclusion and Future Work:

Conclusion: In conclusion the frindings and implications are very useful not only for buisnesses, but many other people such as investors, marketing, and providers. We were able to derive popular items and trends from the charts that could be used to save money and create more opportunites. While the charts also provide room to explore creativity in introducing new ingredients or downsizing to a more minimal approach. Pair this with the recommendation system and you will be more confident in decisions as it will backup the other fidnings while giving people ideas.

Future Directions: This was really and intro to food recommendation. There are many ways to mine data. For example I used both a recommendation algorithm as well as a cosine similarity that I learned in my AI class. Many different methods can give you better insights or maybe personal preference for readings. As we learn more of the intricascies of programming languages, data mining methods, and mathematics we will improve our tools and our ability to create more findings and do it efficiently. After these findings sitting down with the owners we can find methods to increase profitability and customer happiness. For further analysis and data collection I would recommend a more experience data miner that can extarct more or learn more myself as we can build on this project personally. For further data collection studying different years or seasons as well as different pizza shops in other areas around the same city could be interesting. As well as mining geographical data to see how everything changes with populations. Really adding more variables to this data such as customer information could explore more possibilities for insights.